<a href="https://colab.research.google.com/github/JozefSL/pyNotes/blob/main/Kalman/DPR_liqProduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install filterpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 KB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=aac4c9f990804a6e2183c9c561c1fb777c6a6f7693264b8dae955d4ccb795630
  Stored in directory: /root/.cache/pip/wheels/fe/f6/cb/40331472edf4fd399b8cad02973c6acbdf26898342928327fe
Successfully built filterpy


In [3]:
import numpy as np
import pandas as pd
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from filterpy.stats import plot_covariance_ellipse
from filterpy.kalman import predict
from filterpy.kalman import update
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import r2_score 
import ipywidgets as widgets

In [4]:
eiaLgFile = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/49/Eia-logomark.svg/640px-Eia-logomark.svg.png"
eiaLogo = [dict(source = eiaLgFile,
                       x=1.06, y=-0.06,
                       sizex=0.15, sizey=0.15,
                       xanchor="center", yanchor="bottom")]

In [5]:
dprRegions = ['Anadarko Region','Bakken Region','Eagle Ford Region','Niobrara Region','Permian Region', 'Marcellus Region','Utica Region','Appalachia Region','Haynesville Region']
dprR = widgets.Dropdown(options=dprRegions, value='Permian Region', description='DPR Region:', disabled=False)
dprR

Dropdown(description='DPR Region:', index=4, options=('Anadarko Region', 'Bakken Region', 'Eagle Ford Region',…

In [6]:
file = r"https://www.eia.gov/petroleum/drilling/xls/dpr-data.xlsx"
dprRegion = 'Permian Region'
data = pd.read_excel(file, sheet_name=dprR.value, skiprows=2, usecols=[0,1,4]) #index_col=0, , nrows=numRows)
data.columns = ['Month', 'BH', 'PR']
data.head(2)

,Month,BH,PR
0,2007-02-01,253.0,857277.1
1,2007-03-01,242.0,851889.5


In [7]:
data['BH2MF'] = data['BH'].shift(2)
data['dPR'] = data['PR'].diff()
data['dBH2MF'] = data['BH2MF'].diff()

In [8]:
data = data.fillna(method="backfill")
data = data.fillna(method="ffill")

In [9]:
data.tail()

,Month,BH,PR,BH2MF,dPR,dBH2MF
188,2022-10-01,346.0,5487939.9,346.0,32509.0,-3.0
189,2022-11-01,349.0,5531175.0,343.0,43235.1,-3.0
190,2022-12-01,350.0,5569440.2,346.0,38265.2,3.0
191,2023-01-01,350.0,5604943.4,349.0,35503.2,3.0
192,2023-02-01,350.0,5635372.4,350.0,30429.0,1.0


In [12]:
orv = data['PR'].iloc[0]
orr = data['dPR'].iloc[0]

In [13]:
# Step units in months
dt = 1

# x - Origin state estimate vector
x = np.array([orv, orr]).T

# Q - Process noise matrix
Q = Q_discrete_white_noise(dim=2, dt=1., var=.75)

# P - Covariance matrix
P = np.diag([11, 1])

# R - Measurement noise matrix
R = np.diag([1.13, 24])

# H - Measurement function
H = np.array([[1., 0.],
              [0., 1.]])

# F - State transition matrix
F = np.array([[1, dt],
              [0, 1]])

# B - Measurement function
B = np.array([[0.,1],
              [0., 1.7*dt]])

# Measurements for update 
zsu = data[['PR','dPR','BH2MF','dBH2MF']].to_numpy()

# System prediction 
Xp = np.empty((0,2), int)

# System update
Xu = np.empty((0,2), int)
for zs  in zsu:
    z = zs[0:2]
    u = zs[2:4]
 
    u[0]=0
    #u[2]=0
    #print(z)
    x, P = predict(x=x, P=P, F=F, Q=Q, u=u, B=B, alpha=0.99)    #x, P = predict(x=x, P=P, F=F, Q=Q, u=u)
    Xp = np.vstack([Xp, x])

    x, P = update(x, P, z, R, H,)
    Xu = np.vstack([Xu, x])
   
print('R^2:', r2_score(Xp[:,0], data.PR),'|| lastEstimate:', z, '|| lastRigChange:',u)  

R^2: 0.9935224883697922 || lastEstimate: [5635372.4   30429. ] || lastRigChange: [0. 1.]


In [14]:
fig = go.Figure(px.scatter(title="%s liquids production estimates using Kalman filter" % (dprR.value)))
fig.add_trace(go.Scatter(x=data.Month, y=Xp[:,0], mode='markers',name='PredictPR'))
fig.add_trace(go.Scatter(x=data.Month, y=data.PR, mode='markers',name='Reported'))
fig.add_trace(go.Scatter(x=data.Month, y=Xu[:,0], mode='lines+markers',name='UpdatePR'))
fig.layout.images = eiaLogo
fig.show()

In [15]:
fig = go.Figure(px.scatter(title="%s monthly changes in liquids production estimates using Kalman filter" % (dprR.value)))
fig.add_trace(go.Scatter(x=data.Month, y=Xp[:,1], mode='markers',name='PredictPR'))
fig.add_trace(go.Scatter(x=data.Month, y=data.dPR, mode='markers',name='Reported'))
fig.add_trace(go.Scatter(x=data.Month, y=Xu[:,1], mode='lines+markers',name='UpdatePR'))
fig.layout.images = eiaLogo
fig.show()